In [1]:
evals = list()

In [8]:
import utils.common
import evaluation
import importlib
import numpy as np
import time
import models.MatrixFactorization
import models.BPR
import models.ConstraintAutoRec
import models.NeuralLogicRec
importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.MatrixFactorization)
importlib.reload(models.ConstraintAutoRec)
importlib.reload(models.BPR)
importlib.reload(models.NeuralLogicRec)

<module 'models.NeuralLogicRec' from '/mnt/e/owncloud/FH/MasterThesis/Code/models/NeuralLogicRec.py'>

In [12]:
ev = evaluation.Evaluation(utils.common.msd)

In [13]:
model = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.msd['dimensions'], epochs=10, novelty_weight=0.1, diversity_weight=0.1, latent_dim=64)
model.train(utils.common.load_dataset(utils.common.msd, 'train'), utils.common.msd['train']['records'])

Epoch 1/10
3686/3686 [==============================] - 65s 18ms/step - loss: 0.1567 - accuracy: 0.0271
Epoch 2/10
3686/3686 [==============================] - 56s 15ms/step - loss: 0.1394 - accuracy: 0.0314
Epoch 3/10
3686/3686 [==============================] - 58s 16ms/step - loss: 0.1337 - accuracy: 0.0341
Epoch 4/10
 810/3686 [=====>........................] - ETA: 45s - loss: 0.1307 - accuracy: 0.0336

KeyboardInterrupt: 

In [14]:
ev.evaluate_single_thread(model)

{'accuracy': 0.8091957679723775,
 'precision': 1.0,
 'recall': 0.8091957679723775,
 'map@1': 0.16211884779216398,
 'map@5': 0.07913302077329322,
 'map@10': 0.0616346931698607,
 'diversity@5': 0.4069041860033175,
 'diversity@10': 0.4334710502459991,
 'epc@5': 0.9284048515703631,
 'epc@10': 0.9336265989402,
 'epd@5': 0.5377157041415661,
 'coverage@1': 0.21275327771156138,
 'coverage@5': 0.4354886769964243,
 'coverage@10': 0.5713647199046484,
 'name': 'ConstraintAutoRec',
 'dimensions': 6712,
 'latent_dims': 64,
 'accuracy_weight': 1.0,
 'novelty_weight': 0.1,
 'diversity_weight': 0.1,
 'epochs': 10,
 'batch_size': 32,
 'optimizer': 'adam'}

In [43]:
diversity_model = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=30, novelty_weight=0.25, diversity_weight=2.0)
diversity_model.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])


Tensor("args_2:0", shape=(?,), dtype=int64)


ValueError: Shapes must be equal rank, but are 0 and 2 for 'Select' (op: 'Select') with input shapes: [?,10381], [], [?,10381].

In [ ]:
def create_model(novelty_weight):
    m = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=15, novelty_weight=novelty_weight, diversity_weight=0, batch_size=128)
    m.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])
    return m

nov_models2 = [create_model(x) for x in [0.1, 0.5, 1.0, 2.0]]

In [ ]:
import matplotlib.pyplot as plt
models

In [ ]:
#ev = evaluation.Evaluation(utils.common.movie_lens)
#big_eval = ev.evaluate_models(models)
big_eval.head()

In [ ]:
plt.bar(eval_data.index, eval_data['epc@10'])

In [ ]:
plt.plot(eval_data['novelty_weight'], eval_data['epc@10'], 'ro', ls='--')
plt.plot(eval_data['novelty_weight'], eval_data['accuracy'], 'bo', ls='--')
plt.show()

In [ ]:
def plot_bar(data, axis, x_param):
    bottom = data[axis].min() 
    top = data[axis].max()
    margin = (top - bottom) * 0.05
    fig, ax = plt.subplots()
    ax.bar(data.index, data[axis] )
    ax.set_xticks(data.index)
    ax.set_xticklabels(data[x_param])
    ax.set_ylabel(axis)
    ax.set_xlabel(x_param)
    plt.show()

In [2]:
# mf = models.MatrixFactorization.MatrixFactorization(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=10, latent_dim = 64)
# mf.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])

bpr = models.BPR.BPR(utils.common.ml_small['user'], utils.common.movie_lens['dimensions'], epochs=5, latent_dim = 32, batch_size=1024)

In [3]:
bpr.batch_size = 8092
bpr.train(utils.common.load_dataset(utils.common.movie_lens, 'train'), utils.common.movie_lens['train']['records'])

Epoch #0 Loss at step 0: 7.7482
Epoch #1 Loss at step 0: 6.8005
Epoch #2 Loss at step 0: 5.9856
Epoch #3 Loss at step 0: 5.2461
Epoch #4 Loss at step 0: 4.6098


In [127]:
result = ev.evaluate_models([model, bpr])

Evaluating ConstraintAutoRec |============================================================>| 100.0% 
Evaluating BPR |============================================================>| 100.0% 


In [7]:
nlr = models.NeuralLogicRec.NLR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=5, embedding_dim=20, batch_size=32, nr_hidden_layers=4)

In [9]:
nlr.train(utils.common.load_dataset(utils.common.movie_lens), utils.common.movie_lens['train']['records'])
evals.append(ev.evaluate_single_thread(nlr))

ResourceExhaustedError: OOM when allocating tensor with shape[332192,40] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:MatMul] name: neural_logic_rec/likes_estimator/dense_2/Tensordot/MatMul/

In [25]:
ds = utils.common.load_dataset(utils.common.ml_small).take(4)

In [26]:
for record in ds:
    print(record)

{'x': <tf.Tensor 'IteratorGetNext_182790:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_182790:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_182790:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_182790:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_182790:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_182791:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_182791:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_182791:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_182791:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_182791:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_182792:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_182792:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_182792:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNex

{'x': <tf.Tensor 'IteratorGetNext_182990:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_182990:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_182990:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_182990:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_182990:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_182991:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_182991:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_182991:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_182991:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_182991:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_182992:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_182992:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_182992:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNex

{'x': <tf.Tensor 'IteratorGetNext_183192:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_183192:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_183192:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_183192:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_183192:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_183193:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_183193:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_183193:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_183193:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_183193:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_183194:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_183194:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_183194:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNex

{'x': <tf.Tensor 'IteratorGetNext_183368:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_183368:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_183368:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_183368:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_183368:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_183369:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_183369:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_183369:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_183369:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_183369:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_183370:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_183370:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_183370:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNex

{'x': <tf.Tensor 'IteratorGetNext_183546:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_183546:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_183546:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_183546:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_183546:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_183547:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_183547:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_183547:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_183547:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_183547:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_183548:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_183548:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_183548:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNex

...

{'x': <tf.Tensor 'IteratorGetNext_186359:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186359:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186359:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_186359:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_186359:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_186360:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186360:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186360:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_186360:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_186360:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_186361:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186361:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186361:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNex

{'x': <tf.Tensor 'IteratorGetNext_186555:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186555:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186555:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_186555:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_186555:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_186556:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186556:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186556:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_186556:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_186556:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_186557:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186557:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186557:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNex

{'x': <tf.Tensor 'IteratorGetNext_186727:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186727:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186727:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_186727:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_186727:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_186728:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186728:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186728:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_186728:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_186728:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_186729:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186729:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186729:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNex

{'x': <tf.Tensor 'IteratorGetNext_186916:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186916:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186916:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_186916:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_186916:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_186917:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186917:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186917:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNext_186917:4' shape=(10381,) dtype=bool>, 'mask_test': <tf.Tensor 'IteratorGetNext_186917:1' shape=(10381,) dtype=bool>}
{'x': <tf.Tensor 'IteratorGetNext_186918:3' shape=(10381,) dtype=bool>, 'mask': <tf.Tensor 'IteratorGetNext_186918:0' shape=(10381,) dtype=bool>, 'user_id': <tf.Tensor 'IteratorGetNext_186918:2' shape=() dtype=int64>, 'x_test': <tf.Tensor 'IteratorGetNex

KeyboardInterrupt: 